In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import statsmodels.formula.api as smf
import statsmodels

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


# read in data

In [5]:
airbnb_origin2022 = pd.read_csv('listings.csv')
airbnb_origin2022.info() #check data type, the first three columns are redundant information.
airbnb_origin2022.head()

/tmp/ipykernel_911/575434403.py:1: DtypeWarning: Columns (0,2,22,41,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_origin2022 = pd.read_csv('listings.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69358 entries, 0 to 69357
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            69354 non-null  object 
 1   listing_url                                   69357 non-null  object 
 2   scrape_id                                     69357 non-null  object 
 3   last_scraped                                  69357 non-null  object 
 4   source                                        69358 non-null  object 
 5   name                                          69337 non-null  object 
 6   description                                   67839 non-null  object 
 7   neighborhood_overview                         39966 non-null  object 
 8   picture_url                                   69358 non-null  object 
 9   host_id                                       69358 non-null 

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,13913,https://www.airbnb.com/rooms/13913,20220910194334,2022-09-11,city scrape,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730.0,...,4.93,4.75,4.82,NaN,f,2.0,1.0,1.0,0.0,0.20
1,15400,https://www.airbnb.com/rooms/15400,20220910194334,2022-09-11,city scrape,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,60302.0,...,4.82,4.93,4.73,NaN,f,1.0,1.0,0.0,0.0,0.57
2,284532,https://www.airbnb.com/rooms/284532,20220910194334,2022-09-11,city scrape,COSY STUDIO-FLAT WITH A GREAT VIEW,My very RECENTLY RENOVATED flat is located in ...,Shoreditch is simply the coolest area in London!,https://a0.muscache.com/pictures/2932872/00037...,1481412.0,...,4.86,4.72,4.31,NaN,f,1.0,1.0,0.0,0.0,0.34
3,106332,https://www.airbnb.com/rooms/106332,20220910194334,2022-09-11,city scrape,"Lovely large room, Bethnal Green",Lovely characterful room in a house belonging ...,NaN,https://a0.muscache.com/pictures/82437180/20b7...,264345.0,...,4.93,4.92,4.83,NaN,f,1.0,0.0,1.0,0.0,0.47
4,17402,https://www.airbnb.com/rooms/17402,20220910194334,2022-09-11,city scrape,Superb 3-Bed/2 Bath & Wifi: Trendy W1,You'll have a wonderful stay in this superb mo...,"Location, location, location! You won't find b...",https://a0.muscache.com/pictures/39d5309d-fba7...,67564.0,...,4.69,4.88,4.59,NaN,f,5.0,5.0,0.0,0.0,0.36


# clean data namess

In [4]:
# This function will clean up column names in a data frame
# credit by https://gist.github.com/burkesquires/75aeebf4554e9fa427d14354d7fe048c
def clean_dataframe_column_names(df):
    cols = df.columns
    new_column_names = []

    for col in cols:
        new_col = col.lstrip().rstrip().lower().replace (" ", "_") #strip beginning spaces, makes lowercase, add underscore
        new_column_names.append(new_col)

    df.columns = new_column_names

In [5]:
clean_dataframe_column_names(airbnb_origin2022)

In [7]:
airbnb2022 = airbnb_origin2022[['id','host_id','host_listings_count','latitude','longitude','room_type','availability_365','neighbourhood_cleansed','price']
         ]
#rename the columns (orginal is too complex)
cols = airbnb2022.columns.values
airbnb2022.columns = cols

In [8]:
airbnb2022

,id,host_id,host_listings_count,latitude,longitude,room_type,availability_365,neighbourhood_cleansed,price
0,13913,54730,3.0,51.56861,-0.11270,Private room,343,Islington,$50.00
1,15400,60302,1.0,51.48780,-0.16813,Entire home/apt,70,Kensington and Chelsea,$75.00
2,284532,1481412,1.0,51.52851,-0.07426,Entire home/apt,222,Tower Hamlets,$90.00
3,106332,264345,2.0,51.52874,-0.05396,Private room,74,Tower Hamlets,$55.00
4,17402,67564,6.0,51.52195,-0.14094,Entire home/apt,249,Westminster,$379.00
...,...,...,...,...,...,...,...,...,...
69346,9671724,49977905,15.0,51.48300,-0.01346,Private room,0,Greenwich,$55.00
69347,20277958,1132775,1.0,51.50914,-0.10192,Entire home/apt,0,Southwark,$201.00
69348,20512703,1706966,1.0,51.40571,-0.35325,Entire home/apt,4,Richmond upon Thames,$246.00
69349,49698549,34213667,1.0,51.41029,-0.35975,Entire home/apt,2,Richmond upon Thames,$250.00
